In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
import pyspark.sql.functions as F

spark = SparkSession.builder\
        .master('local[*]')\
        .appName("Example") \
        .config("spark.some.config.option","some-value") \
        .getOrCreate()

sc = spark.sparkContext

# Самостоятельная работа

Требуется выяснить:
1. Какое соотношение сторон экрана телефона самое популярное,
2. Плотность пикселей у экрана.

Всего 8 баллов.


# Считывание данных

In [2]:
# Прочитайте содержимое каждого файла в датафрейм и выведите первые несколько строк (1 балл)
# Применив cast к колонке можно изменить тип данных (F.col().cast(IntegerType()).alias())
# Приведите все данные к правильному типу, либо считайе сразу верно (1 балл)
#######
train = spark.read.option(key="header", value="true").csv("train.csv")
test = spark.read.option(key="header", value="true").csv("test.csv")
train.show(3)
test.show(3)

+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|price_range|
+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|          842|   0|        2.2|       0|  1|     0|         7|  0.6|      188|      2|  2|       20|     756|2549|   9|   7|       19|      0|           0|   1|          1|
|         1021|   1|        0.5|       1|  0|     1|        53|  0.7|      136|      3|  6|      905|    1988|2631|  17|   3|        7|      1|           1|   0|          2|
|          563|   1|        0.5|       1|  2|     1|        41|  0.9|      145|      5|  6|     1263|    1716|2603|  11|   2|     

In [3]:
for c in train.columns:
    train=train.withColumn(c, train[c].cast('double'))
train.printSchema()

root
 |-- battery_power: double (nullable = true)
 |-- blue: double (nullable = true)
 |-- clock_speed: double (nullable = true)
 |-- dual_sim: double (nullable = true)
 |-- fc: double (nullable = true)
 |-- four_g: double (nullable = true)
 |-- int_memory: double (nullable = true)
 |-- m_dep: double (nullable = true)
 |-- mobile_wt: double (nullable = true)
 |-- n_cores: double (nullable = true)
 |-- pc: double (nullable = true)
 |-- px_height: double (nullable = true)
 |-- px_width: double (nullable = true)
 |-- ram: double (nullable = true)
 |-- sc_h: double (nullable = true)
 |-- sc_w: double (nullable = true)
 |-- talk_time: double (nullable = true)
 |-- three_g: double (nullable = true)
 |-- touch_screen: double (nullable = true)
 |-- wifi: double (nullable = true)
 |-- price_range: double (nullable = true)



In [4]:
for c in test.columns:
    test=test.withColumn(c, test[c].cast('double'))
test.printSchema()

root
 |-- id: double (nullable = true)
 |-- battery_power: double (nullable = true)
 |-- blue: double (nullable = true)
 |-- clock_speed: double (nullable = true)
 |-- dual_sim: double (nullable = true)
 |-- fc: double (nullable = true)
 |-- four_g: double (nullable = true)
 |-- int_memory: double (nullable = true)
 |-- m_dep: double (nullable = true)
 |-- mobile_wt: double (nullable = true)
 |-- n_cores: double (nullable = true)
 |-- pc: double (nullable = true)
 |-- px_height: double (nullable = true)
 |-- px_width: double (nullable = true)
 |-- ram: double (nullable = true)
 |-- sc_h: double (nullable = true)
 |-- sc_w: double (nullable = true)
 |-- talk_time: double (nullable = true)
 |-- three_g: double (nullable = true)
 |-- touch_screen: double (nullable = true)
 |-- wifi: double (nullable = true)



## Объединим train и test
Найти нужную функцию можно [здесь](https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html)

PS: нужно сделать средствами pyspark (аналог в пандас pd.concat([train, test,], axis=0))

In [5]:
# Объедините train и test (1 балла)
###############
# Ваш код здесь
data = test.union(train)
data.show(3)
data.printSchema()

+---+-------------+----+-----------+--------+----+------+----------+-----+---------+-------+----+---------+--------+------+----+----+---------+-------+------------+----+
| id|battery_power|blue|clock_speed|dual_sim|  fc|four_g|int_memory|m_dep|mobile_wt|n_cores|  pc|px_height|px_width|   ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|
+---+-------------+----+-----------+--------+----+------+----------+-----+---------+-------+----+---------+--------+------+----+----+---------+-------+------------+----+
|1.0|       1043.0| 1.0|        1.8|     1.0|14.0|   0.0|       5.0|  0.1|    193.0|    3.0|16.0|    226.0|  1412.0|3476.0|12.0| 7.0|      2.0|    0.0|         1.0| 0.0|
|2.0|        841.0| 1.0|        0.5|     1.0| 4.0|   1.0|      61.0|  0.8|    191.0|    5.0|12.0|    746.0|   857.0|3895.0| 6.0| 0.0|      7.0|    1.0|         0.0| 0.0|
|3.0|       1807.0| 1.0|        2.8|     0.0| 1.0|   0.0|      27.0|  0.9|    186.0|    3.0| 4.0|   1270.0|  1366.0|2396.0|17.0|10.0|     10.0|    0.0

In [6]:
# Выведите отсортированное распределение плотности пикселей (1 балла)
# в пикселях на дюйм
###############

# Заморочимся но сделаем правильно)): см переведем в дюймы и ррi расчитаем как положено через диагональ


data.withColumn('sc_h', col('sc_h')/2.54)
data.withColumn('sc_w', col('sc_w')/2.54)

data = data.withColumn('ppi', (F.sqrt(F.pow(data.px_width, 2) + F.pow(data.px_height,2)) )/(F.sqrt(F.pow(data.sc_h, 2) + F.pow(data.sc_w, 2))))

data.sort(['ppi'],ascending = True).show()

data.select('ppi').sort(['ppi'],ascending = True).show()

+------+-------------+----+-----------+--------+----+------+----------+-----+---------+-------+-----+---------+--------+------+----+----+---------+-------+------------+----+------------------+
|    id|battery_power|blue|clock_speed|dual_sim|  fc|four_g|int_memory|m_dep|mobile_wt|n_cores|   pc|px_height|px_width|   ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|               ppi|
+------+-------------+----+-----------+--------+----+------+----------+-----+---------+-------+-----+---------+--------+------+----+----+---------+-------+------------+----+------------------+
| 876.0|       1252.0| 1.0|        2.6|     1.0| 1.0|   1.0|      43.0|  0.8|    124.0|    1.0|  3.0|    153.0|   538.0|2447.0|18.0|15.0|     11.0|    1.0|         0.0| 0.0| 23.87173527397815|
| 382.0|       1088.0| 1.0|        3.0|     1.0| 2.0|   1.0|      52.0|  0.3|    164.0|    7.0|  4.0|     86.0|   588.0|1447.0|19.0|16.0|     15.0|    1.0|         1.0| 1.0|23.923839178469603|
| 339.0|       1709.0| 0.0|        

## Сохранение


In [8]:
# Сохраните результат в csv sep=';', encoding='cp1251'
# с колонками id, плотность пикселей и общей инфой в формате "2 ядра 2 гига"
# 2 балла
###############
data_save = data.select('id', 'ppi', 'n_cores', 'int_memory')

data_save.write.csv('results.csv', sep=';', header=True, encoding='cp1251')